In [40]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../../src/')
from model import AEDist
from omegaconf import OmegaConf
import os
import glob
import numpy as np
import torch
from torch import nn
from torch.autograd import grad
adjoint = False
if adjoint:
    from torchdiffeq import odeint_adjoint as odeint
else:
    from torchdiffeq import odeint
import torch.optim as optim
from torch.autograd.functional import jacobian

from geodesic import jacobian, velocity, CondCurve, GeodesicBridgeDensity
from plotly3d.plot import  scatter, trajectories
import torch
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, TensorDataset


In [3]:
wandb.login()
api = wandb.Api()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xingzhis. Use `wandb login --relogin` to force relogin


In [5]:
entity = "xingzhis"
project = "dmae"
# run_id = 'iio2bb24'
# run_id = 'djf8x1o1'
run_id = 'dp90ofk5'
run = api.run(f"{entity}/{project}/{run_id}")
folder_path = '../../src/wandb/'
cfg = OmegaConf.create(run.config)
folder_list = glob.glob(f"{folder_path}*{run.id}*")
ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
ckpt_path = ckpt_files[0]
data_path2 = os.path.join('../', cfg.data.root, cfg.data.name + cfg.data.filetype)
data = np.load(data_path2, allow_pickle=True)
model = AEDist.load_from_checkpoint(ckpt_path)
x_tensor = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
# x_tensor_normalized = model.normalize(x_tensor)

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'activation_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['activation_fn'])`.
  rank_zero_warn(


In [6]:
with torch.no_grad():
    z = model.encode(x_tensor)
    x_recon = model.decode(z)

In [14]:
scatter(z.cpu().numpy(), s=2, colors=data['colors'])

In [15]:
data['colors']

array([1, 2, 2, ..., 0, 0, 3])

In [19]:
np.random.choice(np.where(data['colors'] == 0)[0], 5)

array([ 115, 1766, 1723, 2552, 1223])

In [61]:
np.random.seed(34513)
n = 4
start_ids = np.random.choice(np.where(data['colors'] == 0)[0], n)
start_ids = np.concatenate([start_ids for j in range(4)])
end_ids = np.concatenate([np.random.choice(np.where(data['colors'] == i)[0], n) for i in range(1,5)])
start_batch = torch.tensor(z[start_ids,:]).cpu() 
end_batch = torch.tensor(z[end_ids,:]).cpu() 
dataset = TensorDataset(start_batch, end_batch)
dataloader = DataLoader(dataset, batch_size=len(start_batch), shuffle=True)

/tmp/tmp.bmUrOSbJUg/ipykernel_3528809/2324482316.py:6: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/tmp/tmp.bmUrOSbJUg/ipykernel_3528809/2324482316.py:7: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [62]:
# # Swiss roll random integer
# # np.random.seed(3450)
# # np.random.seed(3451)
# # np.random.seed(3452)
# np.random.seed(345123)
# npts = len(z)
# xs = []

# for j in range(4)
#     start = np.random.choice()
#     # end = np.random.randint(0,npts,size=1)
#     xs.append(torch.tensor(z[start,:]).cpu())
# # xends.append(torch.tensor(z[end,:]).cpu())

# end = np.random.randint(0,npts,size=1)
# # xfin = torch.tensor([[0,0,1]], dtype=torch.float32)
# xfin = torch.tensor(z[end,:]).cpu() 
# xbatch = torch.cat(xs,0)
# endbtch = torch.cat([xfin for i in range(n)],0)

# print(xfin.shape)
# print(xbatch.shape)
# print(endbtch.shape)

# import matplotlib.pyplot as plt
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# scprep.plot.scatter3d(z.detach().cpu().numpy(), ax=ax)
# for i in range(xbatch.shape[0]):
#     scprep.plot.scatter3d(xbatch[i].detach().cpu().numpy(), ax=ax, c='g')
#     # scprep.plot.scatter2d(zhat_no_len_reg[:,i,:].detach().cpu().numpy(), ax=ax)
# scprep.plot.scatter3d(xfin[0].detach().cpu().numpy(), ax=ax, c='r')
# from torch.utils.data import DataLoader, TensorDataset
# dataset = TensorDataset(xbatch, endbtch)
# dataloader = DataLoader(dataset, batch_size=len(xbatch), shuffle=True)

In [63]:
fig = scatter(z.cpu().numpy(), s=2, colors=data['colors'], alpha=0.5)
fig = scatter(start_batch.cpu().numpy(), s=10, fig=fig)
fig = scatter(end_batch.cpu().numpy(), s=10, fig=fig)
fig.show()

In [69]:
gbmodel = GeodesicBridgeDensity(
    func=None,
    data_pts=z,
    input_dim=3, 
    hidden_dim=32, 
    scale_factor=1, 
    symmetric=True, 
    num_layers=3, 
    n_tsteps=1000, 
    lr=1e-3, 
    weight_decay=1e-5,
    density_weight=20.,
    euclidean=True
)
trainer = pl.Trainer(max_epochs=200)
trainer.fit(gbmodel, dataloader)

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning:

The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_env ...

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning:

GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning:

You defined a

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.


In [70]:
x0, x1 = next(iter(dataloader))
with torch.no_grad():
    zhat = gbmodel(x0, x1, gbmodel.ts)

In [71]:
# import torch
# import numpy as np

# # Constants
# n = 20  # Number of longitude lines, adjust as needed
# num_steps = 100  # Number of points per longitude line (time steps)

# # Generate angles
# theta = torch.linspace(0, np.pi / 2, num_steps).unsqueeze(1)  # Shape (100, 1)
# phi = torch.linspace(0, 2 * np.pi, n)  # Shape (n,)

# # Use broadcasting to generate full coordinates
# x = torch.sin(theta) * torch.cos(phi)  # Shape (100, n)
# y = torch.sin(theta) * torch.sin(phi)  # Shape (100, n)
# z = torch.cos(theta) * torch.ones(1, n)  # Shape (100, n)

# # Combine into a single tensor
# longitudelines = torch.stack((x, y, z), dim=-1)  # Final shape (100, n, 3)

# # Check the shape
# # print("Shape of points tensor:", points.shape)


In [73]:
fig = scatter(z.cpu().numpy(), s=2, colors=data['colors'], alpha=0.5)
fig = scatter(x0.detach().cpu().numpy(), s=10, fig=fig)
fig = scatter(x1.detach().cpu().numpy(), s=10, fig=fig)
# fig = trajectories(longitudelines.detach().cpu().numpy(), s=5, c='blue', fig=fig)
fig = trajectories(zhat.detach().cpu().numpy(), s=10, fig=fig, title='Geodesic Bridge (Euclidean, density loss)', filename='heatgeo_denoise_geod_euc.html', rescale=False)
fig.show()